In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

In [2]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [3]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [4]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

In [5]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [6]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [9]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)
mse_loss = keras.losses.MeanSquaredError()

num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 4s 317ms/step - loss: 36.0875 - root_mean_squared_error: 6.0059 - val_loss: 35.2874 - val_root_mean_squared_error: 5.9390
Epoch 2/500
5/5 [==============================] - 0s 11ms/step - loss: 31.5175 - root_mean_squared_error: 5.6127 - val_loss: 33.8097 - val_root_mean_squared_error: 5.8133
Epoch 3/500
5/5 [==============================] - 0s 12ms/step - loss: 32.8320 - root_mean_squared_error: 5.7286 - val_loss: 34.1699 - val_root_mean_squared_error: 5.8441
Epoch 4/500
5/5 [==============================] - 0s 10ms/step - loss: 30.8663 - root_mean_squared_error: 5.5545 - val_loss: 34.7954 - val_root_mean_squared_error: 5.8976
Epoch 5/500
5/5 [==============================] - 0s 13ms/step - loss: 31.9357 - root_mean_squared_error: 5.6500 - val_loss: 32.4266 - val_root_mean_squared_error: 5.6932
Epoch 6/500
5/5 [==============================] - 0s 10ms/step - loss: 35.3877 - root_mean_squared_error: 5.94

In [12]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )

sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[0]
compute_predictions(bnn_model_small)

Predictions mean: 6.26, min: 5.68, max: 6.44, range: 0.77 - Actual: 7.0
Predictions mean: 6.15, min: 5.69, max: 6.39, range: 0.7 - Actual: 6.0
Predictions mean: 5.85, min: 5.09, max: 6.24, range: 1.15 - Actual: 6.0
Predictions mean: 5.34, min: 4.52, max: 5.94, range: 1.42 - Actual: 5.0
Predictions mean: 5.76, min: 5.07, max: 6.11, range: 1.04 - Actual: 6.0
Predictions mean: 5.83, min: 5.18, max: 6.2, range: 1.03 - Actual: 7.0
Predictions mean: 6.25, min: 5.65, max: 6.45, range: 0.8 - Actual: 6.0
Predictions mean: 6.31, min: 5.76, max: 6.46, range: 0.7 - Actual: 7.0
Predictions mean: 5.64, min: 4.89, max: 6.15, range: 1.26 - Actual: 6.0
Predictions mean: 6.22, min: 5.89, max: 6.43, range: 0.53 - Actual: 5.0


In [13]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 2s 38ms/step - loss: 49.8926 - root_mean_squared_error: 7.0631 - val_loss: 48.2899 - val_root_mean_squared_error: 6.9488
Epoch 2/500
17/17 [==============================] - 0s 4ms/step - loss: 46.9064 - root_mean_squared_error: 6.8485 - val_loss: 49.0162 - val_root_mean_squared_error: 7.0009
Epoch 3/500
17/17 [==============================] - 0s 4ms/step - loss: 44.6594 - root_mean_squared_error: 6.6824 - val_loss: 47.6308 - val_root_mean_squared_error: 6.9011
Epoch 4/500
17/17 [==============================] - 0s 4ms/step - loss: 44.6263 - root_mean_squared_error: 6.6800 - val_loss: 42.9542 - val_root_mean_squared_error: 6.5536
Epoch 5/500
17/17 [==============================] - 0s 4ms/step - loss: 43.3768 - root_mean_squared_error: 6.5857 - val_loss: 47.0487 - val_root_mean_squared_error: 6.8589
Epoch 6/500
17/17 [==============================] - 0s 4ms/step - loss: 43.3475 - root_mean_squared_error